In [ ]:
import json
with open("config.json", "rt") as f:
    config_obj = json.load(f)
print(config_obj)
globals().update(config_obj)

{'dataset_path': '<path to your chb-mit directory> e.g ./physionet.org/files/chbmit/1.0.0/', 'clean_data_path': '<path to store clean & aligned dataset> e.g ./data_clean', 'label_output_path': '<path to store generated TOML files> e.g ./ref_labels', 'ignore_lst': ['chb16_18.edf', 'chb16_19.edf', 'chb17c_13.edf', 'chb18_01.edf', 'chb19_01.edf', 'chb11_01.edf', 'chb12_27.edf', 'chb12_28.edf', 'chb12_29.edf', 'chb09_01.edf', 'chb15_01.edf']}


In [ ]:
import os
os.chdir('..')
os.chdir('..')

In [ ]:
# TODO Combined with command-line arguments
with open("global_config.json", "rt") as f:
    config_obj = json.load(f)
# globals().update(config_obj)
dt_fmt = config_obj.datetime_fmt

In [ ]:
from plot.timeline import get_fig_from_obj
from utils.iowrapper import EdfReaderWrapper

In [ ]:
import re # Add TUH-EEG Extra Record Info
import os
import glob
import json
from datetime import datetime, timedelta
import plotly.offline as pyo
import pyedflib


tuh_path = os.path.join(dataset_path, "..", "..", "tuh_eeg", "v2.0.0", "edf") # TODO 使用 latest 链接指代最新版本
plot_args_path = timeline_info_path
if "plotly_fig_path" not in globals(): # TODO 需要回顾一下名称查找规则
    plotly_fig_path = "timeline"
    os.makedirs(plotly_fig_path, exist_ok=True)
# TODO Move plotly.min.js to plot_args_path

def extract_values_from_pattern(pattern):
    regex = r'(\w+)_sz(\d+)_edf(\d+).json'
    match = re.match(regex, pattern)
    
    if match:
        pat = match.group(1)
        sz_cnt = int(match.group(2))
        edf_cnt = int(match.group(3))
        
        return sz_cnt, edf_cnt, pat
    
json_lst = list(os.scandir(plot_args_path))
json_lst.sort(key=lambda fe: extract_values_from_pattern(fe.name)[:2], reverse=True)

pat_2_complete_path = {extract_values_from_pattern(fe.name)[-1]: "" for fe in json_lst}
for first_level_entry in os.scandir(tuh_path):
    if first_level_entry.is_dir():
        for second_level_entry in os.scandir(first_level_entry):
            if second_level_entry.is_dir() and second_level_entry.name in pat_2_complete_path:
                pat_2_complete_path[second_level_entry.name] = second_level_entry.path

for fe in json_lst:
    with open(fe.path, "rt") as f:
        info_obj = json.load(f)
    pat_record_lst = info_obj["record_lst"]
    pat_seizure_lst = info_obj["seizure_lst"]

    fig = get_fig_from_obj(title=fe.name, info_obj=info_obj)

    record_fn_lst =  glob.glob(os.path.join(pat_2_complete_path[extract_values_from_pattern(fe.name)[-1]], "**", "*.edf"), recursive=True)
    pat_complete_lst = []

    for edf_path in record_fn_lst:
        with EdfReaderWrapper(edf_path) as pedf:
            start_dt = pedf.getStartdatetime()
            end_dt = start_dt + timedelta(seconds=(edf_len := pedf.getFileDuration()))
            fs = pedf.getSampleFrequency(0)
            # assert all((FS := pedf.getSampleFrequencies()) == fs) # TODO 支持过滤非脑电数据通道
            pat_complete_lst.append({
                "file": os.path.basename(edf_path), 
                # "span": [start_dt.strftime(dt_fmt), end_dt.strftime(dt_fmt)], 
                "span": [start_dt, end_dt], 
                "info": f"{os.path.basename(edf_path)} of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
            })

    base = [seg["span"][0] for seg in pat_complete_lst]
    length = [(seg["span"][1] - seg["span"][0]).total_seconds() * 1000 for seg in pat_complete_lst]
    text = [seg["info"] for seg in pat_complete_lst]
    color = "LightSeaGreen"

    offset=-8; width=4# ; ymin=offset+width
    fig.add_bar(
        base=base,
        x=length, 
        y=[0] * len(pat_complete_lst),
        hovertext=text, 
        orientation='h', 
        marker=dict(
            color=color,
            # opacity=opacity, 
        ), 
        name="Origin", 
        offset=offset, 
        width=[width] * len(pat_complete_lst), 
    )     
        
    # print(f"Prepare to plot {fe.name}")
    if len(pat_record_lst) != len(pat_complete_lst): print(f"{fe.name[:-4]} non equal!")
    pyo.plot(fig, filename=os.path.join(plotly_fig_path, fe.name[5:-4]+".html"), # include_plotlyjs="./plotly.min.js", 
                auto_open=False, image='svg', image_width=2560, image_height=1440)